In [50]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline,make_pipeline

from sklearn import set_config
set_config(transform_output='pandas')

In [ ]:
df = pd.read_csv('D:\Desktop\Scripts\DNC_deploy_class\data\insurance.csv')
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [ ]:
df_futuro = ({
    'age':[37,40,58,21,43],
    'sex':['male','female','male','female','female'],
    'bmi':[46.43,32.395,28.596,21.89,24.7],
    'children':[3,1,0,2,2],
    'smoker':['no','no','no','no','yes'],
    'region':['southeast','northeast','northwest','southeast','north']
})

In [ ]:
lr = LinearRegression()

In [ ]:
x= df.drop(columns='charges')
y=df['charges']

x_train,x_test,y_train,y_test = train_test_split(x,y,random_state=2023)

In [ ]:
x_train_transform = pd.get_dummies(x_train)

In [ ]:
lr.fit(x_train_transform,y_train)

LinearRegression()

In [ ]:
lr.predict(x_train_transform)

array([ 6297.81640554, 17604.53144584,  8564.16097047, ...,
        8171.64062054, 11728.69287826,  1928.89483764])

In [33]:
encoder = OneHotEncoder(drop='if_binary',handle_unknown='ignore',sparse_output=False)
encoder.fit(x_train)

OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False)

In [39]:
cat_cols = ['sex','smoker','region']
x_train_transform = encoder.fit(x_train[cat_cols])
x_train_transform =x_train_transform.transform(x_train[cat_cols])
x_train_transform.join(x_train.drop(columns=cat_cols))

,sex_male,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,age,bmi,children
632,0.0,0.0,0.0,0.0,1.0,0.0,29,35.530,0
895,0.0,0.0,0.0,0.0,0.0,1.0,61,44.000,0
1173,1.0,0.0,0.0,1.0,0.0,0.0,38,29.260,2
1131,1.0,0.0,0.0,0.0,0.0,1.0,27,45.900,2
363,0.0,0.0,0.0,0.0,0.0,1.0,21,26.400,1
...,...,...,...,...,...,...,...,...,...
884,1.0,0.0,0.0,1.0,0.0,0.0,25,26.695,4
515,1.0,0.0,0.0,0.0,0.0,1.0,58,35.700,0
695,0.0,0.0,0.0,1.0,0.0,0.0,26,40.185,0
454,1.0,0.0,0.0,0.0,1.0,0.0,32,46.530,2


In [46]:
df_futuro=pd.DataFrame(df_futuro)

In [47]:
df_futuro_transform =encoder.transform(df_futuro[cat_cols])
df_futuro_transform.join(df_futuro.drop(columns=cat_cols))

c:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:202: UserWarning: Found unknown categories in columns [2] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,sex_male,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest,age,bmi,children
0,1.0,0.0,0.0,0.0,1.0,0.0,37,46.430,3
1,0.0,0.0,1.0,0.0,0.0,0.0,40,32.395,1
2,1.0,0.0,0.0,1.0,0.0,0.0,58,28.596,0
3,0.0,0.0,0.0,0.0,1.0,0.0,21,21.890,2
4,0.0,1.0,0.0,0.0,0.0,0.0,43,24.700,2


In [48]:
lr.fit(x_train_transform,y_train)

LinearRegression()

In [49]:
lr.predict(df_futuro_transform)

array([ 8843.42655493,  8133.3347356 ,  8742.94662413,  8544.10779522,
       31800.67336424])

In [64]:
encoder = OneHotEncoder(drop='if_binary',handle_unknown='ignore',sparse_output=False)

numeric_processor = make_pipeline(SimpleImputer(strategy='mean'),StandardScaler())
categorical_processor = make_pipeline(SimpleImputer(strategy='most_frequent'),encoder)


preprocess = ColumnTransformer([
    ('encoder',categorical_processor,['sex','smoker','region']),
    ('scale',numeric_processor,['age','bmi','children'])
])
model_pipeline = Pipeline(
    steps=[('preprocess',preprocess),
    ('lr_model',LinearRegression())
])
model_pipeline.fit(x_train,y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('encoder',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sex', 'smoker', 'region']),
                                                 ('scale',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer()),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  ['age', 'bmi',
                                                   'children'])])),
                ('lr_model', LinearRegression())])

In [63]:
model_pipeline.predict(x_test)

array([ 1.16163736e+04,  1.15012201e+04,  2.68177810e+04,  7.05905948e+03,
        7.78191815e+03,  5.01429872e+03,  7.04617329e+03,  2.22023941e+03,
        1.18240459e+04,  3.32418104e+04,  1.21250921e+04,  3.34656405e+04,
        4.01997674e+04,  3.21522892e+04,  7.25481272e+03,  4.01764884e+04,
        1.00259858e+04,  1.12013188e+04,  4.16113446e+03,  3.26169568e+03,
        2.73575744e+04,  2.65622169e+03,  5.93757039e+03,  2.23108761e+03,
        2.18444446e+03,  8.98638483e+03,  2.92518585e+04,  1.15213697e+04,
        3.74926967e+04,  1.40203318e+04,  8.65944669e+03,  9.02188953e+00,
        2.61116275e+04,  4.69659124e+03,  1.11830680e+04,  3.42245754e+04,
        9.79720504e+03,  3.04683155e+04,  1.18948076e+04,  8.22452563e+03,
        8.69321850e+03,  7.90705607e+03,  2.15239531e+03,  7.44095108e+03,
        3.11010610e+04,  1.27207881e+04,  8.77689232e+03,  1.10247871e+04,
        5.75446604e+03,  1.28197810e+04,  4.19645461e+03,  2.56283201e+04,
        3.72611915e+04,  